In [1]:
# Install packages.
!pip install -q scikit-learn xgboost imbalanced-learn joblib matplotlib seaborn pandas


In [ ]:
# Imports and paths 
import os
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
from datetime import datetime

from sklearn.model_selection import train_test_split, StratifiedKFold, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, f1_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import VotingClassifier
from xgboost import XGBClassifier
from imblearn.over_sampling import SMOTE

# Data Path
DATA_PATH = "C:/Users/ntwar/OneDrive/Desktop/BSE-Project/Capstone/SoSens/data/Crop Recommendation using Soil Properties and Weather Prediction.csv"
MODELS_DIR = "models"
OUTPUTS_DIR = "outputs"
RANDOM_STATE = 42
TEST_SIZE = 0.20
N_JOBS = -1

os.makedirs(MODELS_DIR, exist_ok=True)
os.makedirs(OUTPUTS_DIR, exist_ok=True)

print("DATA_PATH =", DATA_PATH)


DATA_PATH = C:/Users/ntwar/OneDrive/Desktop/BSE-Project/Capstone/SoSens/data/Crop Recommendation using Soil Properties and Weather Prediction.csv


In [3]:
# Load data and quick check
df = pd.read_csv(DATA_PATH)
print("Shape:", df.shape)
display(df.head())
print("\nColumns:", df.columns.tolist())


Shape: (3867, 29)


,Soilcolor,Ph,K,P,N,Zn,S,QV2M-W,QV2M-Sp,QV2M-Su,...,PRECTOTCORR-W,PRECTOTCORR-Sp,PRECTOTCORR-Su,PRECTOTCORR-Au,WD10M,GWETTOP,CLOUD_AMT,WS2M_RANGE,PS,label
0,Yellowish brown,5.81,738.231,5.401,0.23,2.976000,13.816,7.993333,10.456667,11.963333,...,2.073333,5.27,12.303333,5.27,3.44,0.73,56.57,6.24,77.03,Barley
1,Yellowish brown,5.43,606.382,10.478,0.23,3.077000,16.421,7.993333,10.456667,11.963333,...,2.073333,5.27,12.303333,5.27,3.44,0.73,56.57,6.24,77.03,Barley
2,brown,5.41,386.580,6.847,0.23,6.611000,16.557,7.993333,10.456667,11.963333,...,2.073333,5.27,12.303333,5.27,3.44,0.73,56.57,6.24,77.03,Barley
3,red,5.65,207.086,3.418,0.23,0.460181,16.075,7.993333,10.456667,11.963333,...,2.073333,5.27,12.303333,5.27,3.44,0.73,56.57,6.24,77.03,Barley
4,red,5.27,317.357,39.282,0.23,2.743000,12.558,7.993333,10.456667,11.963333,...,2.073333,5.27,12.303333,5.27,3.44,0.73,56.57,6.24,77.03,Barley



Columns: ['Soilcolor', 'Ph', 'K', 'P', 'N', 'Zn', 'S', 'QV2M-W', 'QV2M-Sp', 'QV2M-Su', 'QV2M-Au', 'T2M_MAX-W', 'T2M_MAX-Sp', 'T2M_MAX-Su', 'T2M_MAX-Au', 'T2M_MIN-W', 'T2M_MIN-Sp', 'T2M_MIN-Su', 'T2M_MIN-Au', 'PRECTOTCORR-W', 'PRECTOTCORR-Sp', 'PRECTOTCORR-Su', 'PRECTOTCORR-Au', 'WD10M', 'GWETTOP', 'CLOUD_AMT', 'WS2M_RANGE', 'PS', 'label']


In [ ]:
# Detect target column  and show distribution
candidates = [c for c in df.columns if 'crop' in c.lower() or 'label' in c.lower() or 'target' in c.lower() or 'recommend' in c.lower()]
if not candidates:
    raise ValueError("No target column found. Rename column to include 'crop' or 'label' or 'target'. Columns: " + ", ".join(df.columns))
target_col = candidates[0]
print("Selected target column:", target_col)
print("\nTarget counts:")
display(df[target_col].value_counts().head(30))
print("\nNumeric description:")
display(df.describe().T)


Selected target column: label

Target counts:


label
Teff          1260
Maize          732
Wheat          715
Barley         503
Bean           253
Pea             94
Sorghum         72
Dagussa         71
Niger seed      64
Potato          48
Red Pepper      29
Fallow          26
Name: count, dtype: int64


Numeric description:


,count,mean,std,min,25%,50%,75%,max
Ph,3867.0,5.857295,0.676690,4.300000,5.390000,5.780000,6.200000,8.500000
K,3867.0,324.284770,202.250133,41.134000,191.000000,282.000000,405.000000,2119.000000
P,3867.0,11.349588,34.141864,0.000000,2.000000,4.000000,7.920000,782.000000
N,3867.0,0.179153,0.066476,0.000262,0.131200,0.179884,0.230000,0.695600
Zn,3867.0,1.774094,1.460809,0.100000,1.100000,1.500000,2.060000,45.500000
S,3867.0,11.311625,5.542094,0.050000,7.305000,10.700000,14.195500,118.347000
QV2M-W,3867.0,8.346870,0.613948,7.183333,7.933333,8.383333,8.910000,9.723333
QV2M-Sp,3867.0,9.082508,0.790654,7.650000,8.403333,9.340000,9.480000,10.703333
QV2M-Su,3867.0,12.232020,0.894970,10.476667,11.430000,12.166667,12.836667,13.853333
QV2M-Au,3867.0,10.990747,0.993861,9.070000,10.150000,10.926667,11.433333,12.800000


In [ ]:
# Feature engineering (adds N/P, K/P, pH )
df = df.copy()  # work on a copy

# try several common column names mapping
col_map = {}  

# determine available names
cols = [c.lower() for c in df.columns]

# helpers to find a column by substring
def find_col(subs):
    for c in df.columns:
        if subs.lower() in c.lower():
            return c
    return None

col_N = find_col('n')  
col_P = find_col('p')  
col_K = find_col('k')


# Create domain features
if ('N' in df.columns) and ('P' in df.columns):
    df['N_over_P'] = df['N'] / (df['P'] + 1e-6)
elif ('Nitrogen' in df.columns) and ('Phosphorus' in df.columns):
    df['N_over_P'] = df['Nitrogen'] / (df['Phosphorus'] + 1e-6)

if ('K' in df.columns) and ('P' in df.columns):
    df['K_over_P'] = df['K'] / (df['P'] + 1e-6)
elif ('Potassium' in df.columns) and ('Phosphorus' in df.columns):
    df['K_over_P'] = df['Potassium'] / (df['Phosphorus'] + 1e-6)

# checking for column 'Ph' or 'pH'
ph_col = None
if 'Ph' in df.columns:
    ph_col = 'Ph'
elif 'pH' in df.columns:
    ph_col = 'pH'
elif 'pH ' in df.columns:
    ph_col = 'pH '

if ph_col:
    df['pH_bucket'] = pd.cut(df[ph_col], bins=[-10,5.5,6.5,7.5,50], labels=['very_acid','acid','neutral','alkaline'])

# print new columns 
print("New columns added (if any):", [c for c in ['N_over_P','K_over_P','pH_bucket'] if c in df.columns])


New columns added (if any): ['N_over_P', 'K_over_P', 'pH_bucket']


In [ ]:
# Prepare X and y 
# numeric features (all numeric columns except target)
numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
if target_col in numeric_cols:
    numeric_cols.remove(target_col)

features = numeric_cols.copy()

# one-hot encode pH_bucket and small categorical columns (like Soilcolor)
if 'pH_bucket' in df.columns:
    p_dummies = pd.get_dummies(df['pH_bucket'], prefix='pH')
    df = pd.concat([df, p_dummies], axis=1)
    features += p_dummies.columns.tolist()

if 'Soilcolor' in df.columns:
    s_d = pd.get_dummies(df['Soilcolor'], prefix='Soilcolor')
    df = pd.concat([df, s_d], axis=1)
    features += s_d.columns.tolist()

# Ensure features present
features = [f for f in features if f in df.columns]

X = df[features].fillna(df[features].median())
y_raw = df[target_col].astype(str)

le = LabelEncoder()
y = le.fit_transform(y_raw)

print("Feature count:", len(features))
print("Example features:", features[:30])
print("Number of classes:", len(le.classes_))


Feature count: 78
Example features: ['Ph', 'K', 'P', 'N', 'Zn', 'S', 'QV2M-W', 'QV2M-Sp', 'QV2M-Su', 'QV2M-Au', 'T2M_MAX-W', 'T2M_MAX-Sp', 'T2M_MAX-Su', 'T2M_MAX-Au', 'T2M_MIN-W', 'T2M_MIN-Sp', 'T2M_MIN-Su', 'T2M_MIN-Au', 'PRECTOTCORR-W', 'PRECTOTCORR-Sp', 'PRECTOTCORR-Su', 'PRECTOTCORR-Au', 'WD10M', 'GWETTOP', 'CLOUD_AMT', 'WS2M_RANGE', 'PS', 'N_over_P', 'K_over_P', 'pH_very_acid']
Number of classes: 12


In [7]:
# Stratified split and SMOTE on training set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=TEST_SIZE, random_state=RANDOM_STATE, stratify=y)
print("Train:", X_train.shape, "Test:", X_test.shape)

sm = SMOTE(random_state=RANDOM_STATE)
X_train_res, y_train_res = sm.fit_resample(X_train, y_train)
print("After SMOTE, train resampled:", X_train_res.shape)


Train: (3093, 78) Test: (774, 78)
After SMOTE, train resampled: (12096, 78)


In [8]:
# Scale numeric features
scaler = StandardScaler().fit(X_train_res)
X_train_res_s = scaler.transform(X_train_res)
X_test_s = scaler.transform(X_test)


In [9]:
# Tune DecisionTree and SGDClassifier
from sklearn.model_selection import RandomizedSearchCV

# Decision Tree
dt = DecisionTreeClassifier(random_state=RANDOM_STATE)
dt_param_dist = {
    "max_depth": [None, 5, 10, 15, 20],
    "min_samples_split": [2, 4, 8],
    "min_samples_leaf": [1, 2, 4],
    "criterion": ["gini", "entropy"]
}
dt_rs = RandomizedSearchCV(dt, dt_param_dist, n_iter=12, scoring="f1_macro", cv=3, random_state=RANDOM_STATE, n_jobs=N_JOBS, verbose=1)
dt_rs.fit(X_train_res_s, y_train_res)
best_dt = dt_rs.best_estimator_
print("Best DecisionTree params:", dt_rs.best_params_)

# SGDClassifier tuning
sgd = SGDClassifier(random_state=RANDOM_STATE)
sgd_param_dist = {
    'loss': ['log_loss', 'hinge', 'modified_huber'],
    'alpha': [1e-4, 1e-3, 1e-2],
    'penalty': ['l2', 'l1', 'elasticnet'],
    'max_iter': [1000, 2000],
    'learning_rate': ['optimal', 'invscaling']
}
sgd_rs = RandomizedSearchCV(sgd, sgd_param_dist, n_iter=12, scoring="f1_macro", cv=3, random_state=RANDOM_STATE, n_jobs=N_JOBS, verbose=1)
sgd_rs.fit(X_train_res_s, y_train_res)
best_sgd = sgd_rs.best_estimator_
print("Best SGD params:", sgd_rs.best_params_)


Fitting 3 folds for each of 12 candidates, totalling 36 fits
Best DecisionTree params: {'min_samples_split': 2, 'min_samples_leaf': 1, 'max_depth': None, 'criterion': 'gini'}
Fitting 3 folds for each of 12 candidates, totalling 36 fits
Best SGD params: {'penalty': 'l1', 'max_iter': 2000, 'loss': 'log_loss', 'learning_rate': 'optimal', 'alpha': 0.0001}


In [10]:
# KNN baseline and XGBoost tuning
# KNN quick tune
knn = KNeighborsClassifier()
knn_param_dist = {'n_neighbors': [3,5,7,9], 'weights': ['uniform','distance']}
knn_rs = RandomizedSearchCV(knn, knn_param_dist, n_iter=6, scoring="f1_macro", cv=3, random_state=RANDOM_STATE, n_jobs=N_JOBS, verbose=0)
knn_rs.fit(X_train_res_s, y_train_res)
best_knn = knn_rs.best_estimator_
print("Best KNN params:", knn_rs.best_params_)

# XGBoost tuning (compact)
xgb = XGBClassifier(objective='multi:softprob', num_class=len(le.classes_), use_label_encoder=False, eval_metric='mlogloss', random_state=RANDOM_STATE)
xgb_param_dist = {
    'n_estimators': [100,200,300],
    'max_depth': [3,5,7],
    'learning_rate': [0.01, 0.05, 0.1],
    'subsample': [0.7, 0.9, 1.0],
    'colsample_bytree': [0.6, 0.8, 1.0]
}
xgb_rs = RandomizedSearchCV(xgb, xgb_param_dist, n_iter=10, scoring="f1_macro", cv=3, random_state=RANDOM_STATE, n_jobs=N_JOBS, verbose=1)
xgb_rs.fit(X_train_res_s, y_train_res)
best_xgb = xgb_rs.best_estimator_
print("Best XGBoost params:", xgb_rs.best_params_)


Best KNN params: {'weights': 'distance', 'n_neighbors': 3}
Fitting 3 folds for each of 10 candidates, totalling 30 fits
Best XGBoost params: {'subsample': 0.9, 'n_estimators': 200, 'max_depth': 7, 'learning_rate': 0.1, 'colsample_bytree': 1.0}


In [11]:
# Evaluate models
models = {
    "DecisionTree": best_dt,
    "SGD": best_sgd,
    "KNN": best_knn,
    "XGBoost": best_xgb
}

summary = []
for name, model in models.items():
    y_pred = model.predict(X_test_s)
    acc = accuracy_score(y_test, y_pred)
    f1m = f1_score(y_test, y_pred, average='macro')
    print(f"\nModel: {name}  Accuracy: {acc:.4f}  Macro-F1: {f1m:.4f}")
    print(classification_report(y_test, y_pred, target_names=le.classes_, zero_division=0))
    # save confusion matrix figure
    cm = confusion_matrix(y_test, y_pred)
    plt.figure(figsize=(8,6))
    sns.heatmap(cm, annot=False, cmap='viridis')
    plt.title(f"{name} - Confusion matrix")
    plt.tight_layout()
    plt.savefig(os.path.join(OUTPUTS_DIR, f"confusion_{name}.png"))
    plt.close()
    summary.append({'model': name, 'accuracy': acc, 'f1_macro': f1m})

summary_df = pd.DataFrame(summary).sort_values('f1_macro', ascending=False)
display(summary_df)



Model: DecisionTree  Accuracy: 0.3630  Macro-F1: 0.1910
              precision    recall  f1-score   support

      Barley       0.27      0.27      0.27       101
        Bean       0.11      0.12      0.11        51
     Dagussa       0.04      0.07      0.05        14
      Fallow       0.00      0.00      0.00         5
       Maize       0.44      0.46      0.45       146
  Niger seed       0.00      0.00      0.00        13
         Pea       0.15      0.21      0.17        19
      Potato       0.08      0.10      0.09        10
  Red Pepper       0.14      0.17      0.15         6
     Sorghum       0.10      0.14      0.12        14
        Teff       0.53      0.49      0.51       252
       Wheat       0.40      0.34      0.37       143

    accuracy                           0.36       774
   macro avg       0.19      0.20      0.19       774
weighted avg       0.38      0.36      0.37       774


Model: SGD  Accuracy: 0.3307  Macro-F1: 0.2042
              precision    r

,model,accuracy,f1_macro
3,XGBoost,0.468992,0.252517
2,KNN,0.391473,0.214279
1,SGD,0.330749,0.204220
0,DecisionTree,0.363049,0.190999


In [12]:
# Voting ensemble using top 3 models
top3 = list(summary_df['model'].values[:3])
estimators = [(n, models[n]) for n in top3]
print("Top3 models used in ensemble:", top3)

voting = VotingClassifier(estimators=estimators, voting='hard', n_jobs=N_JOBS)
voting.fit(X_train_res_s, y_train_res)
y_pred_v = voting.predict(X_test_s)
acc_v = accuracy_score(y_test, y_pred_v)
f1m_v = f1_score(y_test, y_pred_v, average='macro')
print(f"\nVoting ensemble accuracy: {acc_v:.4f}  Macro-F1: {f1m_v:.4f}")
print(classification_report(y_test, y_pred_v, target_names=le.classes_, zero_division=0))

# save confusion matrix
cm = confusion_matrix(y_test, y_pred_v)
plt.figure(figsize=(10,8))
sns.heatmap(cm, annot=False, cmap='viridis')
plt.title("Voting Ensemble - Confusion matrix")
plt.tight_layout()
plt.savefig(os.path.join(OUTPUTS_DIR, "confusion_voting.png"))
plt.close()


Top3 models used in ensemble: ['XGBoost', 'KNN', 'SGD']

Voting ensemble accuracy: 0.4302  Macro-F1: 0.2412
              precision    recall  f1-score   support

      Barley       0.35      0.44      0.39       101
        Bean       0.25      0.16      0.19        51
     Dagussa       0.10      0.36      0.16        14
      Fallow       0.00      0.00      0.00         5
       Maize       0.46      0.45      0.45       146
  Niger seed       0.00      0.00      0.00        13
         Pea       0.18      0.21      0.20        19
      Potato       0.15      0.20      0.17        10
  Red Pepper       0.20      0.33      0.25         6
     Sorghum       0.12      0.07      0.09        14
        Teff       0.58      0.62      0.60       252
       Wheat       0.51      0.32      0.39       143

    accuracy                           0.43       774
   macro avg       0.24      0.26      0.24       774
weighted avg       0.44      0.43      0.43       774



In [13]:
# save artifacts
joblib.dump(best_dt, os.path.join(MODELS_DIR, "dt_best.pkl"))
joblib.dump(best_sgd, os.path.join(MODELS_DIR, "sgd_best.pkl"))
joblib.dump(best_knn, os.path.join(MODELS_DIR, "knn_best.pkl"))
joblib.dump(best_xgb, os.path.join(MODELS_DIR, "xgb_best.pkl"))
joblib.dump(voting, os.path.join(MODELS_DIR, "voting_ensemble.pkl"))
joblib.dump(scaler, os.path.join(MODELS_DIR, "scaler.pkl"))
joblib.dump(le, os.path.join(MODELS_DIR, "label_encoder.pkl"))
joblib.dump(features, os.path.join(MODELS_DIR, "feature_names.pkl"))

summary_df = summary_df._append({'model': 'VotingEnsemble', 'accuracy': acc_v, 'f1_macro': f1m_v}, ignore_index=True)
summary_df.to_csv(os.path.join(OUTPUTS_DIR, "model_summary.csv"), index=False)
print("Saved models and summary. Models directory:", MODELS_DIR, "Outputs:", OUTPUTS_DIR)


Saved models and summary. Models directory: models Outputs: outputs
